In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [ ]:
all_linked_to_house = []

In [ ]:
for i in range(1, 451):
    page_to_scrape = requests.get(f'https://batdongsan.vn/ban-nha/p{i}')
    soup = BeautifulSoup(page_to_scrape.text, 'lxml')
    houses = soup.find_all('div', class_ = 'datalist')[0].find_all('div', class_ = 'item')
    for house in houses:
        linked_to = house.find('a', class_ = '').get('href')
        all_linked_to_house.append(linked_to)

In [ ]:
pd.DataFrame(all_linked_to_house).to_csv('all_linked_to_house.csv', index = False)

In [ ]:
df_link = pd.read_csv('all_linked_to_house.csv').values.tolist()
df_link_1d = [item for sublist in df_link for item in sublist]

In [ ]:
def change(price):
    if 'tỷ' in price:
        price = price.replace(' tỷ', '')
        new_price = float(price) * int(1e9)
    elif 'triệu' in price:
        price = price.replace(' triệu', '')
        new_price = float(price) * int(1e6)
    else:
        return 0
    return new_price

def extract_info(thong_tin, key, unit):
    start = thong_tin.find(key)
    if start == -1:
        return None
    start += len(key)
    end = thong_tin.find(unit, start)
    if end == -1:
        return None
    value = thong_tin[start:end].strip()
    return value

In [ ]:
prices = []
areas = []
wcs = []
bedrooms = []
des = []

In [ ]:
new_link_list = df_link_1d[3000:3500]

In [ ]:
len(new_link_list)

500

In [ ]:
i = 0
for link in new_link_list:
    scrape_html = requests.get(link)
    soup = BeautifulSoup(scrape_html.text, 'lxml')
    house_info = soup.find_all('div', class_ = 'uk-panel')[1]

    price = house_info.find('strong', class_ = 'price')
    if price == None:
        continue
    price = change(price.text)
    prices.append(price)

    more_info = house_info.find('ul', class_ = 'uk-list').find_all('li')

    a_house = []
    for info in more_info:
        a_house.append(info.text)
    res = '\n'.join(a_house)

    area = extract_info(res, 'Diện tích: ', 'm2')
    areas.append(area)

    wc = extract_info(res, 'Phòng WC: ', 'WC')
    wcs.append(wc)

    bedroom = extract_info(res, 'Phòng ngủ: ', 'PN')
    bedrooms.append(bedroom)

    house_description = soup.find('div', class_ = 'project-global-object-block-003 block-custom').find('div', class_ = 'content')
    des.append(house_description.text.strip())

    i += 1
    print(i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
price_t = prices
area_t = areas
bedroom_t = bedrooms
bathroom_t = wcs

In [ ]:
price_new = [x / 1000000000 for x in price_t]
area_new = [float(x) if x is not None else None for x in area_t]
bedroom_new = [int(x) if x is not None else None for x in bedroom_t]
bathroom_new = [int(x) if x is not None else None for x in bathroom_t]

In [ ]:
import pandas as pd

df = pd.DataFrame({'description': des, 'price': price_new, 'area': area_new, 'bedroom': bedroom_new, 'bathroom': bathroom_new})

In [ ]:
df

,description,price,area,bedroom,bathroom
0,"Mô tả:\r\n+ Nhà 2 mặt hẻm xe hơi thông thoáng,...",5.8,92.0,3.0,3.0
1,"Bán nhà Đội Cấn 58m vuông, 4 tầng gần phố giá ...",6.2,58.0,2.0,2.0
2,"Nhà xây trệt và gác, có 2 phòng ngủ, phòng thờ...",4.1,51.0,2.0,2.0
3,"Nhà đẹp 1/ Lã Xuân Oai,48m2,đường oto,2 PN,cho...",2.5,48.0,2.0,1.0
4,Nhà phố hiện đại ngay Khu du lịch BỬU LONG\n\r...,2200.0,90.0,4.0,2.0
...,...,...,...,...,...
995,"""Bán nhà Bùi Xương Trạch, Thanh Xuân 37m, 5 tầ...",4.3,37.0,4.0,NaN
996,Bán Nhà Lê Quang Định Bình Thạnh. DT: 56m2. LH...,7.9,56.0,6.0,7.0
997,Lợi nhuận 1 TỶ/năm. Sổ đỏ sang tên.\r\n- Cách ...,5.0,80.0,NaN,NaN
998,"Vị trí: Đường Tân Hòa Đông, Quận 6, nút thắt g...",2.0,30.0,1.0,1.0


In [ ]:
df.to_csv('2500_to_3500.csv', index = False)

In [ ]:
df1 = pd.read_csv('4000_to_4500.csv')
df2 = pd.read_csv('4500_to_5000.csv')

In [ ]:
df_merge = pd.concat([df1, df2])

In [ ]:
df_merge.to_csv('4000_to_5000.csv', index = False)

In [ ]:
for i in range(1, 2):
    url = f'https://batdongsan.vn/ban-nha/p{i}'
    page_to_scrape = requests.get(url)
    soup = BeautifulSoup(page_to_scrape.text, 'lxml')
    house_list = soup.find_all('div', class_ = 'datalist')[0]
    houses = house_list.find_all('div', class_ = 'item')
    for house in houses:
        price = house.find('span', class_ = 'price').text.replace('\t', '').replace('\n', '')
        area = house.find('span', class_ = 'acreage')
        linked_to = house.find('a', class_ = '').get('href')
        sub_page = requests.get(linked_to)
        sub_soup = BeautifulSoup(sub_page.text, 'lxml')
        house_description = sub_soup.find('div', class_ = 'content').text
        print(f'Giá BĐS: {price.strip()}')
        if (area == None):
          print(f'Không có thông tin diện tích')
        else:
          print(f'Diện tích: {area.text}')
        print(f'{house_description.strip()}')
        print('')

Giá BĐS: 3899000000 tỷ
Diện tích: 150m2
Mô tả: 
+ Thông số cực đẹp 5 x 30m (150m2)
+ Nhà cấp 4, 2PN, toilet, phòng khách, bếp, sân sau trồng cây.
+ Nhà gần sát Mặt tiền, hẻm nhựa xe hơi tránh.
+ Khu vực tập trung nhiều Khu dân cư, các Dự án lớn, thu hút đông đảo các Nhà Đầu Tư lớn, Huyện Nhà Bè hướng đến mục tiêu trở thành Thành phố trực thuộc TP.Hồ Chí Minh.
+ Giá chưa tới 30tr/m2 bao đầu tư.
+ Sổ đang vay bank 2.2 tỷ, pháp lý chuẩn, chủ cần bán gấp.
☎️   Thiện chí alo em Hiền gặp chính chủ thương lượng, miễn trung gian.

Giá BĐS: 5.5 tỷ
Diện tích: 51m2
Bán nhà HXH Âu Cơ Phường 9 Tân Bình, 51m2 3 Tầng, giá 5 tỷ nhỉnh
+ DT 51m2 cực to cứng nhất khu
+ Kết cấu: 3 Tầng đẹp nhứt nách
+ Hẻm xe hơi mát mẻ an ninh
+ Sổ đẹp hoàn công đủ
+ Hẻm Âu Cơ thông Trần Văn Quang tiện di chuyển đường Bàu Cát, đường Ni Sư Huỳnh Liên, đường Hồng Lạc, Đường Lạc Long Quân, 1 phút ra chợ vải Tân Bình, 5 phút đi qua cư xá Lữ Gia, cư xá Bắc Hải, Quận 10, Quận 1.

Giá BĐS:

In [ ]:
houses = house_list.find_all('div', class_ = 'item')

In [ ]:
test = soup.find('div', class_ = 'datalist')
test_house = test.find('div', class_ = 'item')
linked_to = test_house.find('a', class_ = '').get('href')
sub_page = requests.get(linked_to)
sub_soup = BeautifulSoup(sub_page.text, 'lxml')

In [ ]:
response = requests.get('https://httpbin.org/get')
res_json = response.json()

In [ ]:
print(res_json)

{'args': {}, 'headers': {'Accept': '*/*', 'Accept-Encoding': 'gzip, deflate', 'Host': 'httpbin.org', 'User-Agent': 'python-requests/2.31.0', 'X-Amzn-Trace-Id': 'Root=1-6662a58c-6b795036017c006870824740'}, 'origin': '35.245.129.151', 'url': 'https://httpbin.org/get'}
